In [1]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates


from QUANTAXIS.QAUtil import DATABASE
from QUANTAXIS.QAUtil import  trade_date_sse
from QUANTAXIS.QAUtil.QADate_trade import (
    QA_util_get_pre_trade_date,
    QA_util_get_next_trade_date,
    QA_util_if_tradetime
)

import ind.alpha191 as a191



%load_ext autoreload
%autoreload 2
%aimport tools.Cacher


In [42]:
def pretreate_data(data):
    # returns = smpl.get_current_return(data,'close')
    # returns.name = 'returns'
    # ret_forward = smpl.get_forward_return(data,'close')
    # ret_forward.name = 'ret_forward'
    # # {'Open', 'cap', 'close', 'high', 'ind', 'low', 'returns', 'volume', 'vwap'}
    # data = pd.concat([data, returns, ret_forward], axis=1)
    data = data.assign(vwap=data.amount/(data.volume*100))
    # data = smpl.add_marketvalue_industry(data)
    # data.rename(columns = {"open":"Open",'totalCapital':'cap','industry':'ind'}, inplace=True)
    
    data.rename(columns = {"open":"Open"}, inplace=True)
    data = smpl.add_report_inds(data,'netAssetsPerShare')
    data['pb']=data['close']/data['netAssetsPerShare']
    return data
    # return pd.concat([data, close_ind, vwap_ind, low_ind, high_ind, volume_ind, adv20_ind, adv40_ind, adv81_ind, co_mixed_ind, oh_mixed_ind, lv_mixed_ind], axis=1)




In [16]:
def assemble_stocks_by_codes(codes):
    files = list(map(lambda x:x+'_train_qfq',l))
    return pd.concat(list(map(lambda file:load_cache(file,cache_type=CACHE_TYPE.STOCK),files))).sort_index(level=0)

l = smpl.get_codes_from_blockname('沪深300', sse='all')
df_all =  assemble_stocks_by_codes(l)

In [43]:
df_treated = pretreate_data(df_all)

In [20]:
# ids = [11,24,38,41,42,47,57,69,80,82,83,88,93,97]
ids = [11,24]

# ids = np.arange(1,101)
# np.random.shuffle(ids)

# def generate_alpha_factors(fun_ids, df=None):
#     import ind.alpha191 as a191
#     from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)
#     from base.JuUnits import excute_for_multidates

# for i in fun_ids:
for i in ids:
    fun_name = 'alpha'+ '0'*(3- int(np.log10(i)+1)) +str(i)
    # indx = excute_for_multidates(df, lambda x: getattr(a191, fun_name)(x) ,level=1)
    indx = excute_for_multidates(df_treated, lambda x: getattr(a191, fun_name)(x) ,level=1)
    indx.name = fun_name
    # save_cache('{}_train'.format(fun_name), indx, cache_type=CACHE_TYPE.FACTOR)
    print(indx)
# worker=6
# task = task_chunk_split(ids, worker)
# results = parallal_task(worker, generate_alpha_factors, task, df=df_treated)


code
000001   -4.506804e+07
000002    1.596329e+07
000063   -3.630724e+07
000066    3.796777e+06
000069    2.500154e+07
              ...     
603882   -1.483158e+05
603899   -2.834211e+06
603986    1.431323e+05
603993   -1.358729e+08
605499   -4.952623e+05
Name: alpha011, Length: 248, dtype: float64
code
000001   -0.406365
000002   -0.211400
000063    0.276363
000066    0.133742
000069    0.348468
            ...   
603882    1.635349
603899    1.838936
603986    0.895157
603993   -0.114700
605499   -1.127755
Name: alpha024, Length: 248, dtype: float64


In [50]:
# excute_for_multidates(df_treated.sort_index(), lambda x: a191.alpha011(x) ,level=1)

def alpha001(data, dependencies=['close','Open','volume'], max_window=7):
    # (-1*CORR(RANK(DELTA(LOG(VOLUME),1)),RANK(((CLOSE-OPEN)/OPEN)),6)
    rank_sizenl = np.log(data['volume']).diff(1).rank(axis=0, pct=True)
    rank_ret = (data['close'] / data['Open']) .rank(axis=0, pct=True)
    rel = rank_sizenl.rolling(window=6,min_periods=6).corr(rank_ret) * (-1)
    return rel


alpha001(df_treated.loc[(slice(None),['000001']),:]).to_list()

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 0.1021765230121091,
 -0.15363389800828126,
 -0.14970977234653857,
 0.09345380732159538,
 0.3098907675540282,
 0.7422288963862115,
 0.7429682788557368,
 0.8164834177192828,
 0.798357211912366,
 0.288419309753927,
 -0.37964228176267173,
 -0.5489576055513827,
 -0.6908747292677427,
 -0.8236826450822599,
 -0.8339553596870425,
 -0.8164498706783369,
 -0.8665861057388909,
 -0.7646819276604478,
 -0.7733637982351853,
 -0.7509987838868937,
 -0.7795202212429982,
 -0.7299529505465547,
 -0.3901008267783668,
 -0.5751899641903827,
 -0.5075560390889876,
 -0.48479378996317013,
 -0.4779454872906891,
 -0.5560352297062748,
 -0.40312387942440797,
 0.39393948043222315,
 0.2884349161483486,
 0.4439747561540876,
 0.832818978557414,
 0.4360979753834642,
 0.21355539472009236,
 0.1595266416680219,
 0.2658421854612817,
 0.026344672694508468,
 -0.1361597787024646,
 -0.008629418977376162,
 -0.2292577482760241,
 -0.06638295354798442,
 0.3610980354843532,
 -0.016012901720639534,
 0

In [146]:
# df_all.loc[(slice('2017-01-10'),['000001','000002','000063']),:]
# pd.concat(indxes,axis=1).sort_index().loc[(slice('2017-01-15'),['000001','000002','000063']),:]

# # a = load_cache('alpha11_train', cache_type=CACHE_TYPE.FACTOR)
# # b = load_cache('alpha24_train', cache_type=CACHE_TYPE.FACTOR)
# pd.concat([a,b],axis=1).sort_index().loc[(slice('2017-01-20'),['000001','000002']),:]